In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random

In [ ]:
data = pd.read_csv('datasets/receipts_categorized.csv')
data = data.drop(columns=['Unnamed: 0'])
data

In [ ]:
data_new = pd.read_csv('datasets/receipts_new.csv')
data_new

In [ ]:
grouped_by_receipts = data.groupby(['transaction_key'])
grouped_by_receipts.describe()

In [33]:
def get_helping(data, field):
    freq = dict()
    def fill_freq(x):
        nonlocal freq
        sum_of_receipt = x['line_item_price'].sum()

        for ind, val in enumerate(x[field].values):
            if not np.isnan(val):
                if val in freq:
                    freq[val][0] += 1
                    freq[val][1].append(x['line_item_price'].values[ind] / sum_of_receipt)
                else:
                    freq[val] = [1, [x['line_item_price'].values[ind] / sum_of_receipt]]

    data.groupby(by='transaction_key').apply(fill_freq)
    all = data['transaction_key'].drop_duplicates().shape[0]

    for k in freq.keys():
        freq[k][0] /= all
        freq[k][1] = np.median(freq[k][1])

    data.groupby(by='transaction_key')

    return freq


In [3]:
data = pd.read_csv('out/datasets/data_processed.csv', nrows=500_000)

In [34]:
f = get_helping(data, 'product_id')

In [35]:
f

{0: [8.360592347967853e-05, 0.620911967504632],
 1: [0.001996091423077325, 0.2546551724137931],
 2: [3.135222130487945e-05, 0.23472222222222222],
 3: [0.0013481455161098164, 0.26631432136827426],
 4: [9.405666391463835e-05, 0.03742802303262956],
 5: [0.0018184288356830082, 0.15691728360371784],
 6: [0.00011495814478455799, 0.11280024323502584],
 7: [0.0001358596256544776, 0.18707093821510298],
 8: [0.00034487443435367396, 0.03093145869947276],
 9: [0.0028844043600489097, 0.005025369629122938],
 10: [0.0004807340600081516, 0.042399236948905586],
 11: [0.0017870766143781287, 0.031418876460977754],
 12: [0.00021946554913415616, 0.09304474066253135],
 13: [0.0005225370217479909, 0.047394632922895724],
 14: [1.0450740434959817e-05, 0.02112532820531785],
 15: [0.0044102124635530425, 0.010083213887561714],
 16: [0.002215556972211481, 0.11807473726084423],
 17: [0.0005852414643577498, 0.1184748184221888],
 18: [0.00016721184695935707, 0.036504681014043194],
 19: [0.0004389310982683123, 0.19485

In [ ]:
data_new = pd.read_csv('receipts_new.csv')


In [ ]:
# dupl = data_new.duplicated()
np.where(dupl == True)[0]

In [ ]:
def mul(x):
    m = 1.0
    for i in x:
        m *= i
    return m

dict_product_pos = {}
# Товар: [позиция товара в чеках]

dict_product_freq = {}
# Товар: счетчик появления товара в чеках

def set_pos(x, array_of_prices):
    item = x['line_item_id']
    price = x['product_price']
    if  item not in dict_product_pos:
        dict_product_pos[item] = []
    dict_product_pos[item].append(np.where(array_of_prices == price)[0][0])
    
        
def analyze_receipt(receipt):
    receipt[['line_item_price', 'line_quantity']]
    product_prices = receipt[['line_item_price', 'line_quantity']].apply(lambda x: mul(x), axis=1)
    receipt['product_prices'] = product_prices
#     print(receipt['product_price'])
    sorted_receipt = receipt.sort_values(['product_price'], ascending=False)
    array_of_prices = sorted_receipt['product_price']
    sorted_receipt.apply(lambda x: set_pos(x, array_of_prices), axis=1)
    
#     print(np.where(array_of_prices == ))

keys = data_new['transaction_key'].unique()
for key in keys[:5]:
    receipt = data_new.loc[data_new['transaction_key'] == key]
    analyze_receipt(receipt)

In [ ]:
# dict_product_pos
# np.where(data_new['transaction_key'] in keys[:5])
# keys_group = data_new.groupby(['transaction_key'])
# data_new.loc[data_new['line_item_id'] == 82148960]

data_new.info()

In [ ]:
# cnt = 10
for key in keys_group.groups:
    cnt -= 1
    lines = keys_group.groups[key].array
    receipt = data_new.iloc[lines]
    dupl_receipt = receipt.duplicated(subset=['line_item_id'], keep=False)
#     print(receipt)
    duplicates_indices = dupl_receipt.loc[dupl_receipt == True].index.to_numpy()
    if len(duplicates_indices) != 0:
        data_new.at[duplicates_indices[0], 'line_quantity'] += data_new.loc[duplicates_indices[1:], 'line_quantity'].sum()
        data_new = data_new.drop(duplicates_indices[1:])
        

data_new.info()
    
#     dupl_indices = np.where(dupl_receipt == True)[0]
    
#     if cnt == 0:
#         break
    